In [ ]:
pip uninstall keras
y

In [25]:
import pandas as pd
import numpy as np

# split
from sklearn.model_selection import StratifiedShuffleSplit

# from spam
from collections import Counter

# word embedding
from gensim.models import Word2Vec
import multiprocessing
from tqdm import tqdm

# keras
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import Dense, GlobalMaxPooling1D, Activation, Dropout, GaussianNoise, LSTM, Bidirectional
from keras.layers import Embedding, Input, BatchNormalization, SpatialDropout1D, Conv1D
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [2]:
X_train = pd.read_pickle('X_train.pickle')
X_test = pd.read_pickle('X_test.pickle')

In [3]:
X_train['tokenized_text'] = X_train['tokenized_text'].apply(lambda x: ' '.join(x))
X_test['tokenized_text'] = X_test['tokenized_text'].apply(lambda x: ' '.join(x))

In [4]:
max_features = 100

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train['tokenized_text']))
list_tokenized_train = tokenizer.texts_to_sequences(X_train['tokenized_text'])
list_tokenized_test = tokenizer.texts_to_sequences(X_test['tokenized_text'])

In [6]:
pad_train = pad_sequences(list_tokenized_train, maxlen=300, padding='post')
pad_test = pad_sequences(list_tokenized_test, maxlen=300, padding='post')

In [7]:
vocab_size = len(tokenizer.word_index)+1

In [12]:
embedding_vector = {}
f = open('glove.42B.300d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

1917495it [01:40, 19085.13it/s]


In [13]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 261615/261615 [00:01<00:00, 191489.53it/s]


In [26]:
model = Sequential()
model.add(Embedding(vocab_size,300,weights = [embedding_matrix],input_length=300,trainable = False))
model.add(Bidirectional(LSTM(75)))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
history = model.fit(pad_train,X_train['target'],epochs = 1,batch_size=256,validation_split=0.2)


Train on 1051653 samples, validate on 262914 samples
Epoch 1/5
1051648/1051653 [============================>.] - ETA: 0s - loss: 0.0868 - accuracy: 0.9697

KeyboardInterrupt: 

In [27]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 300)          78484800  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150)               225600    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4832      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 78,715,265
Trainable params: 230,465
Non-trainable params: 78,484,800
_________________________________________________________________
None
